In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from glob import iglob
import re
from datetime import datetime
import random
import cv2
from copy import copy
from importlib import reload
from glob import iglob
import os
import shutil

from src import utils as ut

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!pip install roboflow

from roboflow import Roboflow

API_KEY = "XXXXXXXX"
rf = Roboflow(api_key=API_KEY)

project = rf.workspace("ufsc-onwy0").project("lapix-roadobjsegmentation-v2")
version = project.version(1)
dataset = version.download("coco-segmentation")

In [3]:
from pycocotools.coco import COCO

# cocofile = 'lapix-RoadObjSegmentation-v2-1/train/_annotations.coco.json'
cocofile = 'lapix-RoadObjSegmentation-v2-1/test/_annotations.coco.json'

dirbase = os.path.dirname(cocofile)
coco = COCO(cocofile)
processor = ut.CocoProcessor(coco)
cat_ids = coco.getCatIds()
print(cat_ids)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [4]:
for ix in [*coco.imgs.keys()][:]:
    gt = processor.mask_from_cocoim(ix)
    # print(np.unique(mask))

    bname = coco.imgs[ix]['file_name']
    p = os.path.join(dirbase, bname)
    setname = p.split('/')[-2]
    newp = os.path.join('ds-mocamba-v0.2', setname, 'im', os.path.basename(p))
    ut.create_dirpath_ifneeded(newp)
    shutil.copy(p, newp)
    
    newpgt = ut.get_gtpath(newp)
    ut.create_dirpath_ifneeded(newpgt)
    cv2.imwrite(newpgt, gt)

# Resize images

In [32]:
fpaths = list(iglob('ds-mocamba-v0.2/**/*.jpg', recursive=True))
print(len(fpaths))

357


In [33]:
get_newpath = lambda p: p.replace('ds-mocamba-v0.2/', 'ds-mocamba-v0.2-long1024/')

for p in fpaths[:]:    
    # if os.path.exists(newp):
    #     continue
    
    im = cv2.imread(p, cv2.IMREAD_UNCHANGED)
    newim = ut.resize_im(im)
    newp = get_newpath(p)
    ut.create_dirpath_ifneeded(newp)
    cv2.imwrite(newp, newim)

    pgt = ut.get_gtpath(p)
    gt = cv2.imread(pgt, cv2.IMREAD_UNCHANGED)
    newgt = ut.resize_im(gt, inter_nearest=True)
    newpgt = get_newpath(pgt)
    ut.create_dirpath_ifneeded(newpgt)
    cv2.imwrite(newpgt, newgt)

In [34]:
print("ACABOU!")

ACABOU!
